<a href="https://www.kaggle.com/code/ibkya12/map-0-98-leashmp-competition?scriptVersionId=180607569" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 33.4 MB/s eta 0:00:00


In [2]:
!pip install duckdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.5/18.5 MB 67.7 MB/s eta 0:00:00


## Data Preparation

The training and testing data paths are defined for the .parquet files. We use duckdb to scan search through the large training sets. Just to get started lets sample out an equal number of positive and negatives. 

This query selects an equal number of samples where binds equals 0 (non-binding) and 1 (binding), limited to 30,000 each, to avoid model bias towards a particular class.

In [3]:
import duckdb
import pandas as pd

train_path = '/kaggle/input/leash-predict-chemical-bindings/train.parquet'
test_path = '/kaggle/input/leash-predict-chemical-bindings/test.parquet'

con = duckdb.connect()

df = con.query(f"""(SELECT *
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 0
                        ORDER BY random()
                        LIMIT 150000)
                        UNION ALL
                        (SELECT *
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 1
                        ORDER BY random()
                        LIMIT 150000)""").df()

con.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [4]:
df.head()

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds
0,180247509,O=C(Nc1ccc(Br)c(C(=O)O)c1)OCC1c2ccccc2-c2ccccc21,Cl.Cl.NCCn1cccnc1=O,Nc1ccc(Cl)nc1Cl,O=C(N[Dy])c1cc(Nc2nc(NCCn3cccnc3=O)nc(Nc3ccc(C...,BRD4,0
1,209152004,O=C(Nc1cccc(Br)c1C(=O)O)OCC1c2ccccc2-c2ccccc21,Nc1ncc([N+](=O)[O-])cn1,CC1(C)CCOC1CCN,CC1(C)CCOC1CCNc1nc(Nc2ncc([N+](=O)[O-])cn2)nc(...,sEH,0
2,162686501,O=C(Nc1cc(C(=O)O)cc(C(=O)O)c1)OCC1c2ccccc2-c2c...,Cc1cc(F)ncc1CN,Nc1nc(=O)[nH]cc1Br,Cc1cc(F)ncc1CNc1nc(Nc2cc(C(=O)O)cc(C(=O)N[Dy])...,sEH,0
3,226644704,O=C(Nc1nc2ccc(C(=O)O)cc2s1)OCC1c2ccccc2-c2ccccc21,CC(C)(C)c1ccc(O)c(N)c1,COC(=O)c1sc(-c2ccc(Cl)cc2)cc1N,COC(=O)c1sc(-c2ccc(Cl)cc2)cc1Nc1nc(Nc2nc3ccc(C...,sEH,0
4,101314341,O=C(N[C@@H](Cc1ccc(I)cc1)C(=O)O)OCC1c2ccccc2-c...,Nc1noc2ccc(F)cc12,Cc1cc(N)n(-c2ccccc2)n1,Cc1cc(Nc2nc(Nc3noc4ccc(F)cc34)nc(N[C@@H](Cc3cc...,BRD4,0


## Feature Preprocessing

Lets grab the smiles for the fully assembled molecule `molecule_smiles` and generate ecfps for it. We could choose different radiuses or bits, but 2 and 1024 is pretty standard.

In [5]:
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import OneHotEncoder

# Convert SMILES to RDKit molecules
df['molecule'] = df['molecule_smiles'].apply(Chem.MolFromSmiles)

# Generate ECFPs
def generate_ecfp(molecule, radius=2, bits=1024):
    if molecule is None:
        return None
    return list(AllChem.GetMorganFingerprintAsBitVect(molecule, radius, nBits=bits))

df['ecfp'] = df['molecule'].apply(generate_ecfp)

## Train Model

In [6]:
# One-hot encode the protein_name
onehot_encoder = OneHotEncoder(sparse_output=False)
protein_onehot = onehot_encoder.fit_transform(df['protein_name'].values.reshape(-1, 1))

# Combine ECFPs and one-hot encoded protein_name
X = [ecfp + protein for ecfp, protein in zip(df['ecfp'].tolist(), protein_onehot.tolist())]
y = df['binds'].tolist()

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
df.head()

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds,molecule,ecfp
0,180247509,O=C(Nc1ccc(Br)c(C(=O)O)c1)OCC1c2ccccc2-c2ccccc21,Cl.Cl.NCCn1cccnc1=O,Nc1ccc(Cl)nc1Cl,O=C(N[Dy])c1cc(Nc2nc(NCCn3cccnc3=O)nc(Nc3ccc(C...,BRD4,0,<rdkit.Chem.rdchem.Mol object at 0x7e9c883861f0>,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,209152004,O=C(Nc1cccc(Br)c1C(=O)O)OCC1c2ccccc2-c2ccccc21,Nc1ncc([N+](=O)[O-])cn1,CC1(C)CCOC1CCN,CC1(C)CCOC1CCNc1nc(Nc2ncc([N+](=O)[O-])cn2)nc(...,sEH,0,<rdkit.Chem.rdchem.Mol object at 0x7e9cb7ff2c70>,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,162686501,O=C(Nc1cc(C(=O)O)cc(C(=O)O)c1)OCC1c2ccccc2-c2c...,Cc1cc(F)ncc1CN,Nc1nc(=O)[nH]cc1Br,Cc1cc(F)ncc1CNc1nc(Nc2cc(C(=O)O)cc(C(=O)N[Dy])...,sEH,0,<rdkit.Chem.rdchem.Mol object at 0x7e9cb7ff2ce0>,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,226644704,O=C(Nc1nc2ccc(C(=O)O)cc2s1)OCC1c2ccccc2-c2ccccc21,CC(C)(C)c1ccc(O)c(N)c1,COC(=O)c1sc(-c2ccc(Cl)cc2)cc1N,COC(=O)c1sc(-c2ccc(Cl)cc2)cc1Nc1nc(Nc2nc3ccc(C...,sEH,0,<rdkit.Chem.rdchem.Mol object at 0x7e9cb7ff2d50>,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,101314341,O=C(N[C@@H](Cc1ccc(I)cc1)C(=O)O)OCC1c2ccccc2-c...,Nc1noc2ccc(F)cc12,Cc1cc(N)n(-c2ccccc2)n1,Cc1cc(Nc2nc(Nc3noc4ccc(F)cc34)nc(N[C@@H](Cc3cc...,BRD4,0,<rdkit.Chem.rdchem.Mol object at 0x7e9cb7ff2dc0>,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."


In [8]:
from xgboost import XGBClassifier

In [9]:
xgb = XGBClassifier()

xgb_model = xgb.fit(X_train , y_train)

y_pred_proba = xgb_model.predict_proba(X_test)[:, 1]  # Probability of the positive class

# Calculate the mean average precision
map_score = average_precision_score(y_test, y_pred_proba)
print(f"Mean Average Precision (mAP): {map_score:.2f}")

Mean Average Precision (mAP): 0.97


# Best Score received with CatBoostClassifier

In [10]:
from catboost import CatBoostClassifier
catb_model = CatBoostClassifier().fit(X_train, y_train)
y_pred_proba = catb_model.predict_proba(X_test)[:, 1]  # Probability of the positive class

# Calculate the mean average precision
map_score = average_precision_score(y_test, y_pred_proba)
print(f"Mean Average Precision (mAP): {map_score:.2f}")

Learning rate set to 0.106975
0:	learn: 0.6558379	total: 182ms	remaining: 3m 1s
1:	learn: 0.6243539	total: 328ms	remaining: 2m 43s
2:	learn: 0.6010974	total: 457ms	remaining: 2m 32s
3:	learn: 0.5823888	total: 575ms	remaining: 2m 23s
4:	learn: 0.5642775	total: 712ms	remaining: 2m 21s
5:	learn: 0.5483316	total: 837ms	remaining: 2m 18s
6:	learn: 0.5364527	total: 959ms	remaining: 2m 15s
7:	learn: 0.5273653	total: 1.06s	remaining: 2m 12s
8:	learn: 0.5195131	total: 1.2s	remaining: 2m 12s
9:	learn: 0.5126458	total: 1.33s	remaining: 2m 11s
10:	learn: 0.5053724	total: 1.46s	remaining: 2m 11s
11:	learn: 0.4999587	total: 1.57s	remaining: 2m 9s
12:	learn: 0.4947818	total: 1.69s	remaining: 2m 8s
13:	learn: 0.4905794	total: 1.79s	remaining: 2m 6s
14:	learn: 0.4858274	total: 1.91s	remaining: 2m 5s
15:	learn: 0.4802572	total: 2.04s	remaining: 2m 5s
16:	learn: 0.4764760	total: 2.16s	remaining: 2m 4s
17:	learn: 0.4707022	total: 2.28s	remaining: 2m 4s
18:	learn: 0.4662083	total: 2.4s	remaining: 2m 3s
19:

Look at that Average Precision score. We did amazing! 

Actually no, we just overfit. This is likely recurring theme for this competition. It is easy to predict molecules that come from the same corner of chemical space, but generalizing to new molecules is extremely difficult.

## Test Prediction

 The trained Random Forest model is then used to predict the binding probabilities. These predictions are saved to a CSV file, which serves as the submission file for the Kaggle competition.

In [11]:
import os

# Process the test.parquet file chunk by chunk
test_file = '/kaggle/input/leash-predict-chemical-bindings/test.csv'
output_file = 'submission.csv'  # Specify the path and filename for the output file

# Read the test.parquet file into a pandas DataFrame
for df_test in pd.read_csv(test_file, chunksize=100000):

    # Generate ECFPs for the molecule_smiles
    df_test['molecule'] = df_test['molecule_smiles'].apply(Chem.MolFromSmiles)
    df_test['ecfp'] = df_test['molecule'].apply(generate_ecfp)

    # One-hot encode the protein_name
    protein_onehot = onehot_encoder.transform(df_test['protein_name'].values.reshape(-1, 1))

    # Combine ECFPs and one-hot encoded protein_name
    X_test = [ecfp + protein for ecfp, protein in zip(df_test['ecfp'].tolist(), protein_onehot.tolist())]

    # Predict the probabilities
    probabilities = catb_model.predict_proba(X_test)[:, 1]

    # Create a DataFrame with 'id' and 'probability' columns
    output_df = pd.DataFrame({'id': df_test['id'], 'binds': probabilities})

    # Save the output DataFrame to a CSV file
    output_df.to_csv(output_file, index=False, mode='a', header=not os.path.exists(output_file))
